## Permutation Feature Importance (PFI)

In [70]:
#r "nuget: Microsoft.ML, 5.0.0"
#r "nuget: Microsoft.ML.FastTree, 5.0.0"
#r "nuget: Microsoft.ML.LightGbm, 5.0.0"

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;

Installed Packages Microsoft.ML, 5.0.0 Microsoft.ML.FastTree, 5.0.0 Microsoft.ML.LightGbm, 5.0.0

In [71]:
public class HousingData
{
    [LoadColumn(0)]
    public string City { get; set; }
    [LoadColumn(1)]
    public string Locality { get; set; }
    [LoadColumn(2)]
    public float Bedrooms { get; set; }
    [LoadColumn(3)]
    public float Bathrooms { get; set; }
    [LoadColumn(4)]
    public float SqftLiving { get; set; }
    [LoadColumn(5)]
    public float SqftLot { get; set; }
    [LoadColumn(6)]
    public float Floors { get; set; }
    [LoadColumn(7)]
    public float YearBuilt { get; set; }
    [LoadColumn(8)]
    public float YearRenovated { get; set; }
    [LoadColumn(9)]
    public string PropertyType { get; set; }
    [LoadColumn(10)]
    public float Parking { get; set; }
    [LoadColumn(11)]
    public string Facing { get; set; }
    [LoadColumn(12)]
    public string Lift { get; set; }
    [LoadColumn(13)]
    public string Furnishing { get; set; }
    [LoadColumn(14)]
    public float NearbySchools { get; set; }
    [LoadColumn(15)]
    public float NearbyHospitals { get; set; }
    [LoadColumn(16)]
    public string Society { get; set; }
    [LoadColumn(17)]
    public float Balcony { get; set; }
    [LoadColumn(18)]
    public string Garden { get; set; }
    [LoadColumn(19)] 
    public float Price { get; set; }
}
public class Prediction
{
     [ColumnName("Score")]
    public float Score { get; set; }
}

In [72]:
MLContext mlContext = new MLContext();
IDataView data = mlContext.Data.LoadFromTextFile<HousingData>(path: "HousePrices.csv", hasHeader: true, separatorChar: ',');

In [79]:
// Model pipeline   
string [] columns=new string[]
{
    "Bedrooms","Bathrooms","SqftLiving","SqftLot","Floors",
    "YearBuilt","YearRenovated","Parking",
    "NearbySchools","NearbyHospitals","Balcony"
};

var pipeline = mlContext.Transforms.Concatenate("Features", columns)
.Append(mlContext.Transforms.NormalizeMeanVariance("Features"))
.Append(mlContext.Regression.Trainers.Sdca(labelColumnName: "Price", featureColumnName: "Features"));

In [80]:
// Train and Transform the model
var model=pipeline.Fit(data);
var transformedData = model.Transform(data);

In [81]:
// PFI Implementation
var pfi= mlContext.Regression.PermutationFeatureImportance(
    model.LastTransformer,
    transformedData,
    labelColumnName: "Price"
);

In [86]:
// Display PFI results
pfi.Select((metric, index) => new { index, metric.RSquared })
   .OrderByDescending(f => Math.Abs(f.RSquared.Mean))
   .ToList()
   .ForEach(f =>
   {
       Console.WriteLine($"{Columns[f.index],-20}|\t{f.RSquared.Mean:F6}");      
   }); 



SqftLiving          |	-1.812148
Floors              |	-0.000755
Bathrooms           |	-0.000522
Balcony             |	-0.000456
SqftLot             |	-0.000435
Bedrooms            |	-0.000192
NearbyHospitals     |	-0.000070
NearbySchools       |	-0.000040
Parking             |	0.000034
YearRenovated       |	-0.000016
YearBuilt           |	0.000002
